In [123]:
# https://medium.com/@fractaldle/guide-to-build-faster-rcnn-in-pytorch-95b10c273439
import os
import torch
import torch.nn as nn
from utils import torch_utils

In [90]:
# specify visible GPUs
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = torch_utils.select_device()

using CPU...


In [91]:
net = 'vgg16'
cfg_file = 'cfgs/{}.yml'.format(net);cfg_file

'cfgs/vgg16.yml'

In [92]:
import torch
image = torch.zeros((1, 3, 800, 800)).float()

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # [y1, x1, y2, x2] format
labels = torch.LongTensor([6, 8]) # 0 represents background
sub_sample = 16

In [93]:
import torchvision
dummy_img = torch.zeros((1, 3, 800, 800)).float()
print(dummy_img)
#Out: torch.Size([1, 3, 800, 800])

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])


In [124]:
model = torchvision.models.vgg16(pretrained=True)
fe = list(model.features)
print(fe) # length is 31

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(512, 512, kern

In [125]:
len(fe)

31

In [126]:
req_features = []
k = dummy_img.clone()
for i in fe:
    k = i(k)
    if k.size()[2] < 800//16:
        break
    req_features.append(i)
    out_channels = k.size()[1]
print(len(req_features)) #30
print(out_channels) # 512

30
512


In [11]:
faster_rcnn_fe_extractor = nn.Sequential(*req_features)

In [12]:
out_map = faster_rcnn_fe_extractor(image)
print(out_map.size())

torch.Size([1, 512, 50, 50])


In [23]:
import numpy as np
ratios = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4), dtype=np.float32)

print(anchor_base)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [98]:
ctr_y = (sub_sample-1) / 2.
ctr_x = (sub_sample-1) / 2.

print(ctr_y, ctr_x)
# Out: (8, 8)
for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
        h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
        w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])

        index = i * len(anchor_scales) + j

        anchor_base[index, 0] = ctr_y - (h-1) / 2.
        anchor_base[index, 1] = ctr_x - (w-1) / 2.
        anchor_base[index, 2] = ctr_y + (h-1) / 2.
        anchor_base[index, 3] = ctr_x + (w-1) / 2.
print(anchor_base)

7.5 7.5
[[ -37.254833  -82.50967    52.254833   97.50967 ]
 [ -82.50967  -173.01933    97.50967   188.01933 ]
 [-173.01933  -354.03867   188.01933   369.03867 ]
 [ -56.        -56.         71.         71.      ]
 [-120.       -120.        135.        135.      ]
 [-248.       -248.        263.        263.      ]
 [ -82.50967   -37.254833   97.50967    52.254833]
 [-173.01933   -82.50967   188.01933    97.50967 ]
 [-354.03867  -173.01933   369.03867   188.01933 ]]


In [19]:
import numpy as np
# https://github.com/rbgirshick/py-faster-rcnn/blob/master/lib/rpn/generate_anchors.py
# Verify that we compute the same anchors as Shaoqing's matlab implementation:
#
#    >> load output/rpn_cachedir/faster_rcnn_VOC2007_ZF_stage1_rpn/anchors.mat
#    >> anchors
#
#    anchors =
#
#       -83   -39   100    56
#      -175   -87   192   104
#      -359  -183   376   200
#       -55   -55    72    72
#      -119  -119   136   136
#      -247  -247   264   264
#       -35   -79    52    96
#       -79  -167    96   184
#      -167  -343   184   360

#array([[ -83.,  -39.,  100.,   56.],
#       [-175.,  -87.,  192.,  104.],
#       [-359., -183.,  376.,  200.],
#       [ -55.,  -55.,   72.,   72.],
#       [-119., -119.,  136.,  136.],
#       [-247., -247.,  264.,  264.],
#       [ -35.,  -79.,   52.,   96.],
#       [ -79., -167.,   96.,  184.],
#       [-167., -343.,  184.,  360.]])

def generate_anchors(base_size=16, ratios=[0.5, 1, 2],
                     scales=2**np.arange(3, 6)):
    """
    Generate anchor (reference) windows by enumerating aspect ratios X
    scales wrt a reference (0, 0, 15, 15) window.
    """

    base_anchor = np.array([1, 1, base_size, base_size]) - 1 # [ 0  0 15 15]
    ratio_anchors = _ratio_enum(base_anchor, ratios)
    anchors = np.vstack([_scale_enum(ratio_anchors[i, :], scales)
                         for i in range(ratio_anchors.shape[0])])
    return anchors

def _whctrs(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """

    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    return w, h, x_ctr, y_ctr # 16 16 7.5 7.5

def _mkanchors(ws, hs, x_ctr, y_ctr):
    """
    Given a vector of widths (ws) and heights (hs) around a center
    (x_ctr, y_ctr), output a set of anchors (windows).
    """

    ws = ws[:, np.newaxis] # [23. 16. 11.].T
    hs = hs[:, np.newaxis] # [12. 16. 22.].T
    anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                         y_ctr - 0.5 * (hs - 1),
                         x_ctr + 0.5 * (ws - 1),
                         y_ctr + 0.5 * (hs - 1)))
    """
    [[-3.5  2.  18.5 13. ]
     [ 0.   0.  15.  15. ]
     [ 2.5 -3.  12.5 18. ]]
    """
    return anchors

def _ratio_enum(anchor, ratios):
    """
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """
    #anchor [ 0  0 15 15], ratios [0.5, 1, 2]
    w, h, x_ctr, y_ctr = _whctrs(anchor) # # 16 16 7.5 7.5
    size = w * h # 256
    size_ratios = size / ratios # [512. 256. 128.]
    ws = np.round(np.sqrt(size_ratios)) # [23. 16. 11.]
    hs = np.round(ws * ratios) # [12. 16. 22.]
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    """
    [[-3.5  2.  18.5 13. ]
     [ 0.   0.  15.  15. ]
     [ 2.5 -3.  12.5 18. ]]
    """
    return anchors

def _scale_enum(anchor, scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor)
    ws = w * scales
    hs = h * scales
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors

In [84]:
anchors = generate_anchors();anchors

array([[ -84.,  -40.,   99.,   55.],
       [-176.,  -88.,  191.,  103.],
       [-360., -184.,  375.,  199.],
       [ -56.,  -56.,   71.,   71.],
       [-120., -120.,  135.,  135.],
       [-248., -248.,  263.,  263.],
       [ -36.,  -80.,   51.,   95.],
       [ -80., -168.,   95.,  183.],
       [-168., -344.,  183.,  359.]])

In [76]:
base_size = 16
ratios = [.5, 1, 2]
scales = [8, 16, 32]
base_anchor = np.array([1, 1, base_size, base_size]) - 1
anchors = []
w = base_anchor[2] - base_anchor[0] + 1
h = base_anchor[3] - base_anchor[1] + 1
x_ctr = base_anchor[0] + (w-1)/2
y_ctr = base_anchor[1] + (h-1)/2
size = w*h
for ratio in ratios:
    size_ratio = size/ratio
    ws = np.round(np.sqrt(size_ratio))
    hs = np.round(ws * ratio)
    anchor = np.hstack((x_ctr - 0.5 * (ws - 1),
                 y_ctr - 0.5 * (hs - 1),
                 x_ctr + 0.5 * (ws - 1),
                 y_ctr + 0.5 * (hs - 1)))
    for scale in scales:
        w = anchor[2] - anchor[0] + 1
        h = anchor[3] - anchor[1] + 1
        x_ctr = anchor[0] + (w-1)/2
        y_ctr = anchor[1] + (h-1)/2
        ws = w * scale
        hs = h * scale
        ranchor = np.hstack((x_ctr - 0.5 * (ws - 1),
                 y_ctr - 0.5 * (hs - 1),
                 x_ctr + 0.5 * (ws - 1),
                 y_ctr + 0.5 * (hs - 1)))
        anchors.append(ranchor)
np.vstack(anchors)

array([[ -84.,  -40.,   99.,   55.],
       [-176.,  -88.,  191.,  103.],
       [-360., -184.,  375.,  199.],
       [ -56.,  -56.,   71.,   71.],
       [-120., -120.,  135.,  135.],
       [-248., -248.,  263.,  263.],
       [ -36.,  -80.,   51.,   95.],
       [ -80., -168.,   95.,  183.],
       [-168., -344.,  183.,  359.]])

In [81]:
# https://github.com/rbgirshick/py-faster-rcnn/blob/master/lib/rpn/generate_anchors.py
def Generate_anchors(base_size=16, ratios=[0.5, 1, 2],
                     scales=[8, 16, 32]):
    base_anchor = np.array([1, 1, base_size, base_size]) - 1
    anchors = []
    w, h, x_ctr, y_ctr = whctrs(base_anchor) # 16 16 7.5 7.5
    size = w*h # 256
    for ratio in ratios:
        size_ratio = size/ratio # 512 256 128 
        ws = np.round(np.sqrt(size_ratio)) # 23 16 11
        hs = np.round(ws * ratio) # 12 16 22
        anchor = mkanchors(ws, hs, x_ctr, y_ctr)
        #print(anchor)
        for scale in scales:
            w, h, x_ctr, y_ctr = whctrs(anchor)
            ws = w * scale
            hs = h * scale
            ranchor = mkanchors(ws, hs, x_ctr, y_ctr)
            anchors.append(ranchor)
    return np.vstack(anchors)

In [78]:
def whctrs(anchor):
    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    return w, h, x_ctr, y_ctr # 16 16 7.5 7.5

In [79]:
def mkanchors(ws, hs, x_ctr, y_ctr):

    anchor = np.hstack((x_ctr - 0.5 * (ws - 1),
                         y_ctr - 0.5 * (hs - 1),
                         x_ctr + 0.5 * (ws - 1),
                         y_ctr + 0.5 * (hs - 1)))
    return anchor

In [80]:
print(Generate_anchors())

[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]
 [ -56.  -56.   71.   71.]
 [-120. -120.  135.  135.]
 [-248. -248.  263.  263.]
 [ -36.  -80.   51.   95.]
 [ -80. -168.   95.  183.]
 [-168. -344.  183.  359.]]


In [100]:
fe_size = (800//16)
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)

In [88]:
ctr_x

array([ 16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192, 208,
       224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400, 416,
       432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608, 624,
       640, 656, 672, 688, 704, 720, 736, 752, 768, 784, 800])

In [114]:
ctr = np.zeros((2500,2))
for x in range(len(ctr_x)):
    for y in range(len(ctr_y)):
        ctr[index, 1] = ctr_x[x] - 8
        ctr[index, 0] = ctr_y[y] - 8
        index +=1

In [115]:
ctr

array([[  8.,   8.],
       [ 24.,   8.],
       [ 40.,   8.],
       ...,
       [760., 792.],
       [776., 792.],
       [792., 792.]])

In [116]:
anchors = np.zeros((fe_size * fe_size * 9, 4))
index = 0
for c in ctr:
    ctr_y, ctr_x = c
    for i in range(len(ratios)):
        for j in range(len(anchor_scales)):
            h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
            w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
            anchors[index, 0] = ctr_y - h / 2.
            anchors[index, 1] = ctr_x - w / 2.
            anchors[index, 2] = ctr_y + h / 2.
            anchors[index, 3] = ctr_x + w / 2.
            index += 1
print(anchors.shape)
#Out: [22500, 4]

(22500, 4)


In [119]:
bbox = np.asarray([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=np.float32) # [y1, x1, y2, x2] format
labels = np.asarray([6, 8], dtype=np.int8) # 0 represents background

In [117]:
index_inside = np.where(
        (anchors[:, 0] >= 0) &
        (anchors[:, 1] >= 0) &
        (anchors[:, 2] <= 800) &
        (anchors[:, 3] <= 800)
    )[0]
print(index_inside.shape)

(8940,)


In [120]:
label = np.empty((len(index_inside), ), dtype=np.int32)
label.fill(-1)
print(label.shape)

(8940,)


In [122]:
valid_anchor_boxes = anchors[index_inside]
print(valid_anchor_boxes.shape)

(8940, 4)
